In [1]:
# generate the possible cases of local binding

from itertools import combinations, chain

n_order = 4

# generate all the possible subsets of [n_order]

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# only take the subsets of size >= 2
subsets = list(powerset(range(n_order)))
subsets = [x for x in subsets if len(x) >= 2]

subset2pairs = {}
for subset in subsets:
    pairs = list(combinations(subset, 2))
    subset2pairs[subset] = set(pairs)

pairs_total = list(combinations(range(n_order), 2))
pairs_total_set = set(pairs_total)

# generate the possible cases of local binding
# each possible case is a sequence of subsets
# condition 1: for the k-th subset in each sequence, it should contain at least a pair that is not covered by the previous k-1 subsets
# condition 2: for each sequence, all the pairs should be covered by the subsets

def check_condition1(pairs_remain: set[tuple], subset_new: tuple):
    pairs_new: set[tuple] = subset2pairs[subset_new]
    for pair in pairs_new:
        if pair in pairs_remain:
            return True
    return False

# def check_condition2(pairs_remain: set[tuple], subset_new: tuple):
#     pairs_new: set[tuple] = subset2pairs[subset_new]
#     if pairs_new.issubset(pairs_remain):
#         return True
#     return False

TypeError: 'type' object is not subscriptable

In [ ]:
def generate_cases(n_order: int):
    def recursive_search(current_sequence: list[tuple], pairs_remain: set[tuple]):
        # Base case: all pairs are covered
        if len(pairs_remain) == 0:
            return [(current_sequence, [])]  # Empty coverage for base case
        
        # Try adding each possible subset
        valid_sequences = []
        for subset in subsets:
            current_sequence_set = set(current_sequence)
            # Skip if subset already used
            if subset in current_sequence_set:
                continue
                
            # Check conditions
            if not check_condition1(pairs_remain, subset):
                continue

            # Calculate remaining pairs and newly covered pairs
            pairs_new = subset2pairs[subset]
            newly_covered = pairs_remain.intersection(pairs_new)
            new_pairs_remain = pairs_remain - pairs_new
            
            # Recursively try adding more subsets
            next_results = recursive_search(
                current_sequence + [subset],
                new_pairs_remain
            )
            
            # Add current coverage to each result
            for seq, coverage_history in next_results:
                valid_sequences.append(
                    (seq, [newly_covered] + coverage_history)
                )
            
        return valid_sequences

    # Start recursive search with empty sequence and all pairs
    return recursive_search([], pairs_total_set)

# Example usage:
cases = generate_cases(n_order)
cnt_dep = 0
for sequence, coverage in cases:
    if len(coverage) == len(pairs_total):
        continue
    cnt_dep += 1
    print("Sequence:", sequence)
    print("Coverage history:", coverage)
    print()
print(f"Found {len(cases)} valid sequences")
print(f"Among them, {cnt_dep} cases involve edge dependency")
